# Smart Helmet Accident Detection - Model Training

## Setup Instructions:
1. Upload this notebook to Google Colab
2. Upload your `collected_data` folder (see next cell)
3. Run all cells
4. Download the generated files at the end

---

## Step 1: Upload Your Data

**Instructions:**
1. In the left sidebar, click the folder icon (Files)
2. Create a new folder called `collected_data`
3. Inside it, create two folders: `normal` and `accident`
4. Upload your CSV files:
   - 3 files to `normal/`
   - 3 files to `accident/`

**OR** run this cell to upload via dialog:

In [ ]:
from google.colab import files
import os

# Create directories
os.makedirs('collected_data/normal', exist_ok=True)
os.makedirs('collected_data/accident', exist_ok=True)

print("Upload NORMAL CSV files:")
uploaded = files.upload()
for filename in uploaded.keys():
    os.rename(filename, f'collected_data/normal/{filename}')

print("\nUpload ACCIDENT CSV files:")
uploaded = files.upload()
for filename in uploaded.keys():
    os.rename(filename, f'collected_data/accident/{filename}')

## Step 2: Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import os

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

## Step 3: Load and Prepare Data

In [ ]:
# Configuration
WINDOW_SIZE = 100  # 2 seconds at 50Hz
OVERLAP = 50       # 50% overlap
FEATURES = ['ax', 'ay', 'az', 'gx', 'gy', 'gz']
LABELS = {'normal': 0, 'accident': 1}

def load_data(data_dir='collected_data'):
    """Load all CSV files."""
    all_data = []
    
    for label_name, label_value in LABELS.items():
        label_dir = os.path.join(data_dir, label_name)
        csv_files = [f for f in os.listdir(label_dir) if f.endswith('.csv')]
        print(f"\nLoading {label_name}: {len(csv_files)} files")
        
        for csv_file in csv_files:
            file_path = os.path.join(label_dir, csv_file)
            df = pd.read_csv(file_path)
            df['label'] = label_value
            all_data.append(df)
            print(f"  ✓ {csv_file}: {len(df)} samples")
    
    return pd.concat(all_data, ignore_index=True)

# Load data
df = load_data()
print(f"\n✓ Total samples: {len(df)}")
print(f"\nClass distribution:")
print(df['label'].value_counts())

## Step 4: Create Time Windows

In [ ]:
def create_windows(df, window_size=WINDOW_SIZE, overlap=OVERLAP):
    """Create sliding windows."""
    windows = []
    labels = []
    step = window_size - overlap
    
    for label in df['label'].unique():
        label_data = df[df['label'] == label][FEATURES].values
        
        for i in range(0, len(label_data) - window_size + 1, step):
            window = label_data[i:i + window_size]
            if len(window) == window_size:
                windows.append(window)
                labels.append(label)
    
    return np.array(windows), np.array(labels)

X, y = create_windows(df)
print(f"✓ Created {len(X)} windows")
print(f"  Shape: {X.shape}")
print(f"  Normal: {np.sum(y == 0)}")
print(f"  Accident: {np.sum(y == 1)}")

## Step 5: Normalize Data

In [ ]:
# Normalize
n_samples, n_timesteps, n_features = X.shape
X_reshaped = X.reshape(-1, n_features)

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X_reshaped)
X_normalized = X_normalized.reshape(n_samples, n_timesteps, n_features)

print(f"✓ Data normalized")
print(f"  Mean: {X_normalized.mean():.4f}")
print(f"  Std: {X_normalized.std():.4f}")

## Step 6: Train/Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.2, random_state=42, stratify=y
)

print(f"✓ Split complete")
print(f"  Training: {len(X_train)} windows")
print(f"  Testing: {len(X_test)} windows")

## Step 7: Build Model

In [ ]:
model = keras.Sequential([
    layers.Input(shape=(WINDOW_SIZE, len(FEATURES))),
    layers.Conv1D(16, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(32, kernel_size=3, activation='relu'),
    layers.GlobalAveragePooling1D(),
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

## Step 8: Train Model

In [ ]:
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.2,
    verbose=1
)

# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

## Step 9: Evaluate Model

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✓ Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"✓ Test Loss: {test_loss:.4f}")

## Step 10: Convert to TensorFlow Lite

In [ ]:
# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save TFLite model
with open('accident_model.tflite', 'wb') as f:
    f.write(tflite_model)

print(f"✓ TFLite model size: {len(tflite_model) / 1024:.2f} KB")

## Step 11: Generate C Header File

In [ ]:
# Convert to C array
hex_array = ', '.join([f'0x{byte:02x}' for byte in tflite_model])

header_content = f"""// Auto-generated TensorFlow Lite model
// Smart Helmet Accident Detection Model

#ifndef ACCIDENT_MODEL_H
#define ACCIDENT_MODEL_H

const unsigned int accident_model_len = {len(tflite_model)};
const unsigned char accident_model[] = {{
  {hex_array}
}};

#endif  // ACCIDENT_MODEL_H
"""

with open('accident_model.h', 'w') as f:
    f.write(header_content)

print("✓ C header file generated")

## Step 12: Save Scaler Parameters

In [ ]:
import json

scaler_params = {
    'mean': scaler.mean_.tolist(),
    'scale': scaler.scale_.tolist()
}

with open('scaler_params.json', 'w') as f:
    json.dump(scaler_params, f, indent=2)

print("✓ Scaler parameters saved")

## Step 13: Download Files

Run this cell to download all generated files:

In [ ]:
from google.colab import files

print("Downloading files...")
files.download('accident_model.tflite')
files.download('accident_model.h')
files.download('scaler_params.json')

print("\n✅ ALL DONE!")
print(f"\nModel Performance: {test_accuracy * 100:.2f}% accuracy")
print(f"Model Size: {len(tflite_model) / 1024:.2f} KB")
print("\nNext: Copy accident_model.h to your ESP32 project!")